# Verify AWS credentials propagate to cluster workers

NOTE: This should only really be necessary in a cloud compute environment where cluster 
worker threads/nodes are spawned via kubernetes.  Clusters on our HPC hardware will not
need to specfically send credentials to workers. 

In [ ]:
import os
import logging

def myAWS_Credentials():
    """Test function to return AWS credential information."""
    return {
    "AWS_PROFILE": os.environ.get("AWS_PROFILE", "<not set>"),
    "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID', '<not set>')
}


##### V V V V V V V V V V V V V V V V ##### set the profile

os.environ['AWS_PROFILE'] = "default"  

##### ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 


1) Establish AWS credentials.

In [ ]:
os.environ['AWS_PROFILE'] = 'esip-qhub'
%run ./Help_AWS_Credentials.ipynb
## Sets AWS_* secrets based on contents of $HOME/.aws/credentials

2. What credentials are set in the execution environment?

In [ ]:
myAWS_Credentials()  ## should return the key for the above-named profile

3. Start a default cluster

In [ ]:
## pick the correct helper for the environment where you are running...
%run ./Start_Dask_Cluster_Nebari.ipynb
# %run ./Start_Dask_Cluster_Nebari.ipynb
# %run ./Start_Dask_Cluster_Denali.ipynb
# %run ./Start_Dask_Cluster_Tallgrass.ipynb
# %run ./Start_Dask_Cluster_PangeoCHS.ipynb


4. Force a worker to report the credentials on their node

In [ ]:
client.submit(myAWS_Credentials).result() 
## This runs the myAWS_Credentials function on a remote node worker (not in this notebook).

5. Shut down client and cluster resources.

In [ ]:
client.close(); cluster.close()